# Gravimetry Electronic Lab Book

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

## Theory
The period of oscillation of a pendulum $T = 2 \pi \sqrt{\frac{L}{g}}$

So, we get $ g = \frac{4 \pi^2 L}{T^2}$. This works best if we have a very light pendulum, (here the calculation assumes the pendulum is massles). so using a string or thin wire is best. This also assumes small angles, and no air resistance.

## Experiment Procedures
The simplest way would be to tie a mass to a string, measure the length (and uncertainty) then let it oscillate by displacing it w/ small angles. Repeat this for different string lengths.

We can record the oscillation using a phone, then analyzing the footage with tracker. For calibration, we can use a meter ruler in the same plane as the oscillation.

## Error Analysis
For direct measurements, such as that of the length of the string and the period we use the usual formulas for mean and standard deviation stats.mean(array) for the mean, stats.tstd(array) for the standard deviation.

### $g$ Error Propogation
here, we have $ \sigma_g^2 = (\frac{-8L \pi^2}{T^3})^2 \sigma_T^2 + (\frac{4\pi^2}{T^2}) ^2\sigma_L^2$

This was derived using the error propogation discussed in mathematical techniques class. It could be worth considering,after calculating g and it's standard deviation for each trial, we weight each value of g with $\frac{1}{\sigma_g^2}$ since if the standard deviation is lower, the uncertainty is lower.

In [3]:
#function for returning mean and standard deviation of an array
def mean_std(a):
    return stats.mean(a),stats.tstd(a)
#given arrays of L, and T, return array of g
def calc_g(L,T):
    return (4*np.power(np.pi,)*L)/(np.power(T,2))
#calculate the standard deviation of g
def g_std(mean_l,std_l,mean_t,std_t):
    a = np.power((-8*mean_l,np.power(np.pi,2))/(np.power(mean_t,3)),2)
    b = np.power((4*np.power(np.pi,2))/(np.power(mean_t,2)))
    return np.sqrt(a*np.power(std_t,2)+b*np.power(std_l))